In [63]:
import pandas as pd
import polars as pl
from datetime import datetime

from factorlib.utils.system import get_data_dir

In [96]:
raw_data_dir = get_data_dir() / 'raw'
prices = pl.scan_csv(raw_data_dir / 'prices_wrds.csv').collect(streaming=True)
prices = (
    prices.lazy()
    .select(
        pl.col('datadate').alias('date_index'),
        pl.col('tic').alias('ticker'),
        (pl.col('prccd').cast(pl.Float64) / pl.col('ajexdi').cast(pl.Float64)).alias('price')
    )
    .collect(streaming=True)
)
prices

date_index,ticker,price
str,str,f64
"""1990-01-02""","""AIR""",24.0
"""1990-01-03""","""AIR""",24.166667
"""1990-01-04""","""AIR""",23.666667
"""1990-01-05""","""AIR""",23.916667
"""1990-01-08""","""AIR""",23.5
"""1990-01-09""","""AIR""",23.416667
"""1990-01-10""","""AIR""",23.416667
"""1990-01-11""","""AIR""",22.916667
"""1990-01-12""","""AIR""",22.5


In [97]:
training_tickers = pl.scan_csv(raw_data_dir / 'tickers_to_train.csv').collect(streaming=True)
training_tickers = training_tickers.select(pl.col('ticker')).to_series().to_list()
training_prices = prices.lazy().filter(pl.col('ticker').is_in(training_tickers)).collect(streaming=True)
len(training_prices.select(pl.col('ticker').unique()))

1152

In [98]:
training_prices_unstacked = training_prices.to_pandas().set_index(['date_index', 'ticker']).unstack('ticker').droplevel(0, axis=1).reset_index()
training_prices_unstacked = pl.from_pandas(training_prices_unstacked)
training_prices_unstacked

date_index,AAME,AAON,AATC,AB,ABC,ABCB,ABM,ABMD,ABT,ACFN,ACGL,ACIW,ACU,ACUR,ADBE,ADC,ADM,ADP,ADSK,ADTN,AE,AEE,AEGN,AEIS,AEM,AEO,AEP,AES,AET,AEY,AFAM,AFG,AFL,AGCO,AGM,AGX,…,WEYS,WFC,WHR,WINT,WIRE,WLY,WLYB,WM,WMB,WNC,WOR,WPP,WRB,WRI,WRLD,WSBC,WSFS,WSO,WST,WTM,WTRG,WTS,WTT,WVFC,WVVI,WWR,X,XCRA,XOM,XOMA,XRAY,Y,YELL,ZBRA,ZEUS,ZION,ZIXI
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""1990-01-02""",2.5,null,null,3.84375,null,null,4.1875,5.875,8.65625,null,null,null,4.25,312.5,1.265625,null,8.538125,6.234375,5.03125,null,3.5,28.625,8.375,null,8.25,null,33.125,null,14.5,null,6.4075,18.5,1.175,null,null,69.34033,…,3.069444,2.828125,32.25,null,null,3.3125,3.296875,1.656,6.625,null,10.166667,54.0625,1.624143,13.944444,null,10.166667,2.5,2.610192,4.71875,35.25,2.150439,19.125,null,null,null,inf,null,7.875788,12.5,7159.090909,1.041667,60.038266,271250.0,null,null,3.5,7.333333
"""1990-01-03""",2.5,null,null,4.03125,null,null,4.234375,5.8125,8.6875,null,null,null,4.375,306.25,1.328125,null,8.538125,6.203125,5.15625,null,3.5,28.875,8.5,null,8.25,null,33.0,null,14.4375,null,6.4075,18.416667,1.15,null,null,69.34033,…,2.944444,2.703125,32.375,null,null,3.25,3.296875,1.688,6.770833,null,10.277778,55.0,1.64853,14.055556,null,10.0,2.5,2.57492,4.6875,35.75,2.131238,19.875,null,null,null,inf,null,7.875788,12.375,7575.757576,1.041667,59.708386,273750.0,null,null,3.5625,7.333333
"""1990-01-04""",2.5,null,null,4.0,null,null,4.140625,5.625,8.671875,null,null,null,4.5,303.15,1.390625,null,8.491722,6.171875,5.03125,null,3.5,28.75,8.5,null,8.375,null,32.5,null,14.46875,null,6.875,18.666667,1.125,null,null,69.34033,…,2.972222,2.671875,32.75,null,null,3.34375,3.296875,1.625,6.5625,null,10.277778,57.5,1.633898,14.0,null,10.0,2.541667,2.57492,4.75,36.0,2.131238,19.75,null,null,null,inf,null,7.689769,12.25,7878.787879,1.0,60.038266,268750.0,null,null,3.5625,7.2
"""1990-01-05""",2.375,null,null,4.0,null,null,4.140625,5.75,8.578125,null,null,null,4.5,290.65,1.421875,null,8.120499,6.125,5.03125,null,3.5,28.125,8.125,null,8.5,null,31.75,null,14.375,null,6.875,18.75,1.125,null,null,69.34033,…,3.027778,2.59375,31.75,null,null,3.25,3.296875,1.688,6.583333,null,10.277778,60.0,1.633898,14.0,null,10.0,2.5,null,4.8125,35.625,2.150439,19.125,null,null,null,inf,null,8.625863,12.1875,7727.272727,1.0625,60.038266,265000.0,null,null,3.625,7.2
"""1990-01-08""",2.375,null,null,4.0,null,null,4.09375,5.75,8.578125,null,null,null,4.5,290.65,1.4375,null,8.213305,6.140625,4.984375,null,3.75,28.125,8.125,null,8.375,null,31.625,null,14.4375,null,6.875,18.75,1.141667,null,null,69.34033,…,3.125,2.671875,32.25,null,null,3.34375,3.265625,1.688,6.666667,null,10.277778,59.375,1.62902,14.0,null,10.5,2.5,null,4.8125,36.0,2.169639,19.375,null,null,null,inf,null,8.625863,12.375,7878.787879,1.020833,60.368147,262500.0,null,null,3.5625,7.466667
"""1990-01-09""",2.375,null,null,3.9375,null,null,4.140625,5.625,8.546875,null,null,null,4.5,287.5,1.40625,null,8.213305,6.03125,4.96875,null,3.75,28.25,8.125,null,8.25,null,31.5,null,14.5,null,6.875,18.75,1.133333,null,null,68.410795,…,3.055556,2.59375,32.25,null,null,3.25,3.296875,1.656,6.541667,null,10.222222,59.375,1.599757,14.0,null,10.0,2.416667,2.57492,4.84375,37.0,2.188839,19.375,null,null,null,inf,null,8.625863,12.125,7348.484848,1.104167,60.368147,260000.0,null,null,3.625,7.466667
"""1990-01-10""",2.375,null,null,3.84375,null,null,4.125,5.5,8.4375,null,null,null,4.375,287.5,1.375,null,8.074096,5.96875,4.96875,null,3.5,28.125,8.25,null,8.5,null,31.625,null,14.125,null,6.875,18.166667,1.091667,null,null,68.410795,…,3.138889,2.625,32.375,null,null,3.3359375,3.296875,1.656,6.291667,null,10.277778,60.3125,1.546106,13.888889,null,10.25,2.416667,2.610192,4.78125,36.25,2.131238,19.5,null,null,null,inf,null,8.2

In [125]:
prices_resampled = training_prices_unstacked.to_pandas().set_index('date_index')
prices_resampled.index = pd.to_datetime(prices_resampled.index)
new_index = pd.date_range(start=prices_resampled.index.min(), end=prices_resampled.index.max())
new_index
prices_resampled = prices_resampled.reindex(new_index)
prices_resampled
prices_resampled.ffill(inplace=True)
prices_resampled = pl.from_pandas(prices_resampled.reset_index())
prices_resampled = prices_resampled.select(pl.col('index').alias('date_index'), pl.all().exclude('index'))
prices_resampled

date_index,ABC,ABT,ACGL,ADBE,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AJG,ALB,ALK,ALL,AME,AMGN,AON,AOS,APA,APD,APH,ATO,AVB,AXP,AZO,BA,BAC,BAX,BDX,BEN,BIIB,BIO,BK,BMY,…,SNPS,SO,SPG,SPGI,STE,STT,STZ,SWKS,SYK,T,TECH,TER,TRMB,TXN,TYL,UHS,UNH,UNP,VLO,VRTX,VTRS,VZ,WAB,WAT,WDC,WEC,WELL,WFC,WHR,WM,WMB,WRB,WST,XOM,XRAY,ZBRA,ZION
datetime[ns],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1990-01-02 00:00:00,null,8.65625,null,1.265625,8.538125,6.234375,5.03125,28.625,33.125,null,1.175,177.818979,6.1875,null,5.34375,null,1.981481,1.072917,12.555556,0.722222,7.521645,12.375,null,11.833333,null,11.625,null,20.5,11.65625,12.75,7.828125,1.611111,null,5.416667,5.415916,14.25,…,null,14.5625,null,7.25,null,4.9375,0.555556,1.166667,1.546875,16.1875,0.089875,2.71875,null,2.3671875,null,1.171875,0.3828125,9.96875,3.78125,null,3.518519,28.5,null,null,4.3125,10.666667,14.75,2.828125,32.25,1.656,6.625,1.624143,4.71875,12.5,1.041667,null,3.5
1990-01-03 00:00:00,null,8.6875,null,1.328125,8.538125,6.203125,5.15625,28.875,33.0,null,1.15,177.608293,6.15625,null,5.28125,null,2.0,1.098958,12.518519,0.715278,7.62987,12.375,null,11.583333,null,11.75,null,21.0,11.75,13.0625,7.75,1.657407,null,5.5,5.498728,14.53125,…,null,14.625,null,7.203125,null,5.0,0.555556,1.125,1.59375,15.90625,0.089875,2.71875,null,2.3125,null,1.234375,0.375,9.828125,3.84375,null,3.388889,28.4375,null,null,4.5625,10.458333,14.75,2.703125,32.375,1.688,6.770833,1.64853,4.6875,12.375,1.041667,null,3.5625
1990-01-04 00:00:00,null,8.671875,null,1.390625,8.491722,6.171875,5.03125,28.75,32.5,null,1.125,175.080061,6.15625,null,5.25,null,1.981481,1.119792,12.333333,0.722222,7.19697,12.21875,null,11.333333,null,11.625,null,20.916667,11.5,12.8125,7.6875,1.694444,null,5.5,5.465603,14.4375,…,null,14.0,null,7.34375,null,4.953125,0.555556,1.125,1.59375,15.5625,0.089875,2.75,null,2.234375,null,1.21875,0.371094,9.78125,3.90625,null,3.351852,27.34375,null,null,4.625,10.416667,15.0,2.671875,32.75,1.625,6.5625,1.633898,4.75,12.25,1.0,null,3.5625
1990-01-05 00:00:00,null,8.578125,null,1.421875,8.120499,6.125,5.03125,28.125,31.75,null,1.125,169.812911,6.15625,null,5.34375,null,1.962963,1.135417,12.074074,0.715278,6.980519,12.375,null,11.666667,null,11.5,null,20.625,11.25,12.625,7.671875,1.740741,null,5.5,5.482166,14.0,…,null,13.75,null,7.265625,null,4.96875,0.576389,1.166667,1.609375,14.8125,0.0976875,2.65625,null,2.203125,null,1.1875,0.355469,9.78125,3.9375,null,3.351852,26.71875,null,null,4.75,10.25,14.875,2.59375,31.75,1.688,6.583333,1.633898,4.8125,12.1875,1.0625,null,3.625
1990-01-06 00:00:00,null,8.578125,null,1.421875,8.120499,6.125,5.03125,28.125,31.75,null,1.125,169.812911,6.15625,null,5.34375,null,1.962963,1.135417,12.074074,0.715278,6.980519,12.375,null,11.666667,null,11.5,null,20.625,11.25,12.625,7.671875,1.740741,null,5.5,5.482166,14.0,…,null,13.75,null,7.265625,null,4.96875,0.576389,1.166667,1.609375,14.8125,0.0976875,2.65625,null,2.203125,null,1.1875,0.355469,9.78125,3.9375,null,3.351852,26.71875,null,null,4.75,10.25,14.875,2.59375,31.75,1.688,6.583333,1.633898,4.8125,12.1875,1.0625,null,3.625
1990-01-07 00:00:00,null,8.578125,null,1.421875,8.120499,6.125,5.03125,28.125,31.75,null,1.125,169.812911,6.15625,null,5.34375,null,1.962963,1.135417,12.074074,0.715278,6.980519,12.375,null,11.666667,null,11.5,null,20.625,11.25,12.625,7.671875,1.740741,null,5.5,5.482166,14.0,…,null,13.75,null,7.265625,null,4.96875,0.576389,1.166667,1.609375,14.8125,0.0976875,2.65625,null,2.203125,null,1.1875,0.355469,9.78125,3.9375,null,3.351852,26.71875,null,null,4.75,10.25,14.875,2.59375,31.75,1.688,6.583333,1.633898,4.8125,12.1875,1.0625,null,3.625
1990-01-08 00:00:00,null,8.578125,null,1.4375,8.213305,6.140625,4.984375,28.125,31.625,null,1.141667,168.759481,6.15625,null,5.375,null,1.9

In [126]:
# sanity check to ensure tradeable tickers have values for their last dates as they should be in the current sp500
tradeable_tickers = pl.scan_csv(raw_data_dir / 'tickers_to_trade.csv').select(pl.col('ticker')).collect(streaming=True).to_series().to_list()
training_prices_unstacked = prices_resampled.lazy().select(pl.col('date_index'), pl.col(tradeable_tickers)).filter(pl.col('date_index').is_between(datetime(1990, 1, 1), datetime(2023, 6, 1), closed='left')).collect(streaming=True)
training_prices_unstacked

date_index,ABC,ABT,ACGL,ADBE,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AJG,ALB,ALK,ALL,AME,AMGN,AON,AOS,APA,APD,APH,ATO,AVB,AXP,AZO,BA,BAC,BAX,BDX,BEN,BIIB,BIO,BK,BMY,…,SNPS,SO,SPG,SPGI,STE,STT,STZ,SWKS,SYK,T,TECH,TER,TRMB,TXN,TYL,UHS,UNH,UNP,VLO,VRTX,VTRS,VZ,WAB,WAT,WDC,WEC,WELL,WFC,WHR,WM,WMB,WRB,WST,XOM,XRAY,ZBRA,ZION
datetime[ns],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1990-01-02 00:00:00,null,8.65625,null,1.265625,8.538125,6.234375,5.03125,28.625,33.125,null,1.175,177.818979,6.1875,null,5.34375,null,1.981481,1.072917,12.555556,0.722222,7.521645,12.375,null,11.833333,null,11.625,null,20.5,11.65625,12.75,7.828125,1.611111,null,5.416667,5.415916,14.25,…,null,14.5625,null,7.25,null,4.9375,0.555556,1.166667,1.546875,16.1875,0.089875,2.71875,null,2.3671875,null,1.171875,0.3828125,9.96875,3.78125,null,3.518519,28.5,null,null,4.3125,10.666667,14.75,2.828125,32.25,1.656,6.625,1.624143,4.71875,12.5,1.041667,null,3.5
1990-01-03 00:00:00,null,8.6875,null,1.328125,8.538125,6.203125,5.15625,28.875,33.0,null,1.15,177.608293,6.15625,null,5.28125,null,2.0,1.098958,12.518519,0.715278,7.62987,12.375,null,11.583333,null,11.75,null,21.0,11.75,13.0625,7.75,1.657407,null,5.5,5.498728,14.53125,…,null,14.625,null,7.203125,null,5.0,0.555556,1.125,1.59375,15.90625,0.089875,2.71875,null,2.3125,null,1.234375,0.375,9.828125,3.84375,null,3.388889,28.4375,null,null,4.5625,10.458333,14.75,2.703125,32.375,1.688,6.770833,1.64853,4.6875,12.375,1.041667,null,3.5625
1990-01-04 00:00:00,null,8.671875,null,1.390625,8.491722,6.171875,5.03125,28.75,32.5,null,1.125,175.080061,6.15625,null,5.25,null,1.981481,1.119792,12.333333,0.722222,7.19697,12.21875,null,11.333333,null,11.625,null,20.916667,11.5,12.8125,7.6875,1.694444,null,5.5,5.465603,14.4375,…,null,14.0,null,7.34375,null,4.953125,0.555556,1.125,1.59375,15.5625,0.089875,2.75,null,2.234375,null,1.21875,0.371094,9.78125,3.90625,null,3.351852,27.34375,null,null,4.625,10.416667,15.0,2.671875,32.75,1.625,6.5625,1.633898,4.75,12.25,1.0,null,3.5625
1990-01-05 00:00:00,null,8.578125,null,1.421875,8.120499,6.125,5.03125,28.125,31.75,null,1.125,169.812911,6.15625,null,5.34375,null,1.962963,1.135417,12.074074,0.715278,6.980519,12.375,null,11.666667,null,11.5,null,20.625,11.25,12.625,7.671875,1.740741,null,5.5,5.482166,14.0,…,null,13.75,null,7.265625,null,4.96875,0.576389,1.166667,1.609375,14.8125,0.0976875,2.65625,null,2.203125,null,1.1875,0.355469,9.78125,3.9375,null,3.351852,26.71875,null,null,4.75,10.25,14.875,2.59375,31.75,1.688,6.583333,1.633898,4.8125,12.1875,1.0625,null,3.625
1990-01-06 00:00:00,null,8.578125,null,1.421875,8.120499,6.125,5.03125,28.125,31.75,null,1.125,169.812911,6.15625,null,5.34375,null,1.962963,1.135417,12.074074,0.715278,6.980519,12.375,null,11.666667,null,11.5,null,20.625,11.25,12.625,7.671875,1.740741,null,5.5,5.482166,14.0,…,null,13.75,null,7.265625,null,4.96875,0.576389,1.166667,1.609375,14.8125,0.0976875,2.65625,null,2.203125,null,1.1875,0.355469,9.78125,3.9375,null,3.351852,26.71875,null,null,4.75,10.25,14.875,2.59375,31.75,1.688,6.583333,1.633898,4.8125,12.1875,1.0625,null,3.625
1990-01-07 00:00:00,null,8.578125,null,1.421875,8.120499,6.125,5.03125,28.125,31.75,null,1.125,169.812911,6.15625,null,5.34375,null,1.962963,1.135417,12.074074,0.715278,6.980519,12.375,null,11.666667,null,11.5,null,20.625,11.25,12.625,7.671875,1.740741,null,5.5,5.482166,14.0,…,null,13.75,null,7.265625,null,4.96875,0.576389,1.166667,1.609375,14.8125,0.0976875,2.65625,null,2.203125,null,1.1875,0.355469,9.78125,3.9375,null,3.351852,26.71875,null,null,4.75,10.25,14.875,2.59375,31.75,1.688,6.583333,1.633898,4.8125,12.1875,1.0625,null,3.625
1990-01-08 00:00:00,null,8.578125,null,1.4375,8.213305,6.140625,4.984375,28.125,31.625,null,1.141667,168.759481,6.15625,null,5.375,null,1.9

In [127]:
training_returns = training_prices_unstacked.lazy().select((pl.col('date_index'), pl.all().exclude('date_index').pct_change())).collect(streaming=True)
training_returns = training_returns.select(pl.col('date_index'), pl.all().exclude('date_index').cast(pl.Float64))
training_returns

date_index,ABC,ABT,ACGL,ADBE,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AJG,ALB,ALK,ALL,AME,AMGN,AON,AOS,APA,APD,APH,ATO,AVB,AXP,AZO,BA,BAC,BAX,BDX,BEN,BIIB,BIO,BK,BMY,…,SNPS,SO,SPG,SPGI,STE,STT,STZ,SWKS,SYK,T,TECH,TER,TRMB,TXN,TYL,UHS,UNH,UNP,VLO,VRTX,VTRS,VZ,WAB,WAT,WDC,WEC,WELL,WFC,WHR,WM,WMB,WRB,WST,XOM,XRAY,ZBRA,ZION
datetime[ns],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1990-01-02 00:00:00,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1990-01-03 00:00:00,null,0.00361,null,0.049383,0.0,-0.005013,0.024845,0.008734,-0.003774,null,-0.021277,-0.001185,-0.005051,null,-0.011696,null,0.009346,0.024272,-0.00295,-0.009615,0.014388,0.0,null,-0.021127,null,0.010753,null,0.02439,0.008043,0.02451,-0.00998,0.028736,null,0.015385,0.015291,0.019737,…,null,0.004292,null,-0.006466,null,0.012658,0.0,-0.035714,0.030303,-0.017375,0.0,0.0,null,-0.023102,null,0.053333,-0.020408,-0.014107,0.016529,null,-0.036842,-0.002193,null,null,0.057971,-0.019531,0.0,-0.044199,0.003876,0.019324,0.022013,0.015015,-0.006623,-0.01,0.0,null,0.017857
1990-01-04 00:00:00,null,-0.001799,null,0.047059,-0.005435,-0.005038,-0.024242,-0.004329,-0.015152,null,-0.021739,-0.014235,0.0,null,-0.005917,null,-0.009259,0.018957,-0.014793,0.009709,-0.056738,-0.012626,null,-0.021583,null,-0.010638,null,-0.003968,-0.021277,-0.019139,-0.008065,0.022346,null,0.0,-0.006024,-0.006452,…,null,-0.042735,null,0.019523,null,-0.009375,0.0,0.0,0.0,-0.021611,0.0,0.011494,null,-0.033784,null,-0.012658,-0.010417,-0.004769,0.01626,null,-0.010929,-0.038462,null,null,0.013699,-0.003984,0.016949,-0.011561,0.011583,-0.037322,-0.030769,-0.008876,0.013333,-0.010101,-0.04,null,0.0
1990-01-05 00:00:00,null,-0.010811,null,0.022472,-0.043716,-0.007595,0.0,-0.021739,-0.023077,null,0.0,-0.030084,0.0,null,0.017857,null,-0.009346,0.013953,-0.021021,-0.009615,-0.030075,0.012788,null,0.029412,null,-0.010753,null,-0.013944,-0.021739,-0.014634,-0.002033,0.027322,null,0.0,0.00303,-0.030303,…,null,-0.017857,null,-0.010638,null,0.003155,0.0375,0.037037,0.009804,-0.048193,0.086926,-0.034091,null,-0.013986,null,-0.025641,-0.042105,0.0,0.008,null,0.0,-0.022857,null,null,0.027027,-0.016,-0.008333,-0.02924,-0.030534,0.038769,0.003175,0.0,0.013158,-0.005102,0.0625,null,0.017544
1990-01-06 00:00:00,null,0.0,null,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,null,0.0,null,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0,null,0.0,null,0.0,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,…,null,0.0,null,0.0,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0,null,0.0,0.0,0.0,0.0,null,0.0,0.0,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0
1990-01-07 00:00:00,null,0.0,null,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,null,0.0,null,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0,null,0.0,null,0.0,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,…,null,0.0,null,0.0,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0,null,0.0,0.0,0.0,0.0,null,0.0,0.0,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0
1990-01-08 00:00:00,null,0.0,null,0.010989,0.011429,0.002551,-0.009317,0.0,-0.003937,null,0.014815,-0.006203,0.0,null,0.005848,null,-0.009434,-0.002294,-0.018405,0.009709,0.062016,0.010101,null,0.007143,null,-0.007246,null,0.012121,0.005556,0.009901,0.00611,0.00266,null,0.007576,-0.006042,0.006696,…,null,0.027273,null,-0.010753,null,0.006289,0.0,0.0,0.019417,0.016878,0.0,0.011765,null,0.014184,null,0.0,-0.021978,0.004792,-0.015873,null,0.01105,0.008187,null,null,0.0,-0.00813,0.0,0.03012,0.015748,0.0,0.012658,-0.002985,0.0,0.015385,-0.039216,nul

In [128]:
training_returns.write_csv(raw_data_dir / 'training_returns.csv')